# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO - DOTA 2

In [ ]:
from modules.downloader import FileDownloader
from modules.extractor import ZipExtractor
import shutil
import json

path_download   = os.path.join("data","external")
link_download   = "https://github.com/odota/dotaconstants/archive/refs/heads/master.zip"

FD              = FileDownloader()
status, path    = FD.download(link_download, path_download)
if status:
    # Define the path to the downloaded zip file
    path_extract= os.path.join(os.path.dirname(path), "dotaconstants")
    EX          = ZipExtractor(path, path_extract)
    dir_extract = EX.extract()
    # Check if the extraction directory contains files
    if len(os.listdir(dir_extract)) > 0:
        # Define the path to the extracted files and the destination directory
        path_extract_build  = os.path.join(dir_extract, os.listdir(dir_extract)[0], "build")
        path_to             = os.path.join("constants","games","dota2")
        # If the extraction path exists, copy the files to the destination directory
        if os.path.exists(path_extract_build):
            os.makedirs(path_to, exist_ok=True)
            # Copy files from the extraction path to the destination directory
            for item in os.listdir(path_extract_build):
                src = os.path.join(path_extract_build, item)
                dst = os.path.join(path_to, item)
                if os.path.isfile(src):
                    shutil.copy2(src, dst)
            # Verificar se todos os arquivos foram copiados com sucesso
            arquivos_copiados   = set(os.listdir(path_to))
            arquivos_esperados  = set(os.listdir(path_extract_build))
            if arquivos_copiados == arquivos_esperados:
                # Excluir a pasta path_extract e o arquivo path
                shutil.rmtree(path_extract)
                os.remove(path)
            else:
                print("Alguns arquivos não foram copiados:", arquivos_esperados - arquivos_copiados)

# INSERINDO NO BANCO DE DADOS

BASE IMPORTS

In [ ]:
from modules.downloader import FileDownloader
from modules import db_manager
from jinja2 import Template
from tqdm import tqdm
import json

FD                              = FileDownloader()
repository_img                  = "https://cdn.cloudflare.steamstatic.com"

VERSÕES

In [ ]:
versions                            = {}
path_file                           = os.path.join("constants", "games", "dota2", "patch.json")
path_insert                         = "repository\\querys\\operation\\insert\\game_versions.sql"

# Read the SQL query template from the file
with open(path_insert, "r", encoding="utf-8") as file:
    query_insert        = file.read()

# Read the JSON file containing game versions
if os.path.exists(path_file):
    with open(path_file, "r", encoding="utf-8") as file:
        versions        = json.load(file)

if versions:
    for version in tqdm(versions, desc="Inserting Game Versions"):
        sql_template    = Template(query_insert)
        rendered_sql    = sql_template.render(
            ID          = version["id"]                                     if "id"             in version else None,
            NAME        = version["name"].replace("'", "").strip()          if "name"           in version else None,
            DATE        = version["date"].replace("'", "").strip()          if "date"           in version else None,
            ID_GAME     = 1,
        ).replace("'None'", "NULL").replace("None", "NULL")\
        .replace("'False'", "FALSE").replace("False", "FALSE")\
        .replace("'True'", "TRUE").replace("True", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
        except Exception as e:
            print(f"Error inserting video data: {e}")
            db_manager.connection.rollback()

HEROES E DESCRIÇÃO

In [ ]:
herois                          = {}
herois_description              = {}
path_file                       = os.path.join("constants", "games", "dota2", "heroes.json")
path_file2                      = os.path.join("constants", "games", "dota2", "hero_lore.json")
path_insert                     = "repository\\querys\\operation\\insert\\heroes.sql"
path_download_img               = os.path.join("data", "processed", "dota2", "heroes", "img")
path_download_icon              = os.path.join("data", "processed", "dota2", "heroes", "icon")

# Read the JSON file containing game versions
if os.path.exists(path_file):
    with open(path_file, "r", encoding="utf-8") as file:
        herois                  = json.load(file)

# Read the JSON file containing hero descriptions
if os.path.exists(path_file2):
    with open(path_file2, "r", encoding="utf-8") as file:
        herois_description      = json.load(file)

# Read the SQL query template from the file
with open(path_insert, "r", encoding="utf-8") as file:
    query_insert        = file.read()

if herois and herois_description:
    for id_heroi in tqdm(herois, desc="Inserting Herois the Game"):
        heroi                   = herois[id_heroi]
        sql_template            = Template(query_insert)
        path_download_img_final = os.path.join(path_download_img,"{}.png".format(os.path.splitext(os.path.basename(heroi["img"]))[0]))
        path_download_icon_final= os.path.join(path_download_icon,"{}.png".format(os.path.splitext(os.path.basename(heroi["icon"]))[0]))
        rendered_sql            = sql_template.render(
            ID_HERO             = heroi["id"]                                           if "id"                 in heroi else None,
            NAME                = heroi["name"].replace("'", "").strip()                if "name"               in heroi else None,
            ID_GAME             = 1,
            DESCRIPTION         = herois_description[heroi["name"].replace("npc_dota_hero_","")].replace("'", "").strip() if heroi["name"].replace("npc_dota_hero_","") in herois_description else "",
            PRIMARY_ATTR        = heroi["primary_attr"].replace("'", "").strip()        if "primary_attr"       in heroi else None,
            ATTACK_TYPE         = heroi["attack_type"].replace("'", "").strip()         if "attack_type"        in heroi else None,
            ROLES               = "'"+"', '".join([row.replace("'", "").strip() for row in heroi["roles"]])+"'" if "roles" in heroi else None,
            IMG                 = path_download_img_final,
            ICON                = path_download_icon_final,
            BASE_HEALTH         = heroi["base_health"]                                  if "base_health"        in heroi else None,
            BASE_HEALTH_REGEN   = heroi["base_health_regen"]                            if "base_health_regen"  in heroi else None,
            BASE_MANA           = heroi["base_mana"]                                    if "base_mana"          in heroi else None,
            BASE_MANA_REGEN     = heroi["base_mana_regen"]                              if "base_mana_regen"    in heroi else None,
            BASE_ARMOR          = heroi["base_armor"]                                   if "base_armor"         in heroi else None,
            BASE_MR             = heroi["base_mr"]                                      if "base_mr"            in heroi else None,
            BASE_ATTACK_MIN     = heroi["base_attack_min"]                              if "base_attack_min"    in heroi else None,
            BASE_ATTACK_MAX     = heroi["base_attack_max"]                              if "base_attack_max"    in heroi else None,
            BASE_STR            = heroi["base_str"]                                     if "base_str"           in heroi else None,
            BASE_AGI            = heroi["base_agi"]                                     if "base_agi"           in heroi else None,
            BASE_INT            = heroi["base_int"]                                     if "base_int"           in heroi else None,
            STR_GAIN            = heroi["str_gain"]                                     if "str_gain"           in heroi else None,
            AGI_GAIN            = heroi["agi_gain"]                                     if "agi_gain"           in heroi else None,
            INT_GAIN            = heroi["int_gain"]                                     if "int_gain"           in heroi else None,
            ATTACK_RANGE        = heroi["attack_range"]                                 if "attack_range"       in heroi else None,
            PROJECTILE_SPEED    = heroi["projectile_speed"]                             if "projectile_speed"   in heroi else None,
            ATTACK_RATE         = heroi["attack_rate"]                                  if "attack_rate"        in heroi else None,
            BASE_ATTACK_TIME    = heroi["base_attack_time"]                             if "base_attack_time"   in heroi else None,
            ATTACK_POINT        = heroi["attack_point"]                                 if "attack_point"       in heroi else None,
            MOVE_SPEED          = heroi["move_speed"]                                   if "move_speed"         in heroi else None,
            TURN_RATE           = heroi["turn_rate"]                                    if "turn_rate"          in heroi else None,
            CM_ENABLED          = heroi["cm_enabled"]                                   if "cm_enabled"         in heroi else None,
            LEGS                = heroi["legs"]                                         if "legs"               in heroi else None,
            DAY_VISION          = heroi["day_vision"]                                   if "day_vision"         in heroi else None,
            NIGHT_VISION        = heroi["night_vision"]                                 if "night_vision"       in heroi else None,
            LOCALIZED_NAME      = heroi["localized_name"].replace("'", "").strip()      if "localized_name"     in heroi else None
        ).replace("'None'", "NULL").replace("None", "NULL")\
        .replace("'False'", "FALSE").replace("False", "FALSE")\
        .replace("'True'", "TRUE").replace("True", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
        except Exception as e:
            print(f"Error inserting video data: {e}")
            db_manager.connection.rollback()
        
        # BAIXANDO IMAGEM DO HERÓI
        status, path    = FD.download((repository_img + heroi["img"]), path_download_img_final)
        
        # BAIXANDO ICONE DO HERÓI
        status, path    = FD.download((repository_img + heroi["icon"]), path_download_icon_final)


ITEMS

In [ ]:
items                           = {}
herois_description              = {}
path_file                       = os.path.join("constants", "games", "dota2", "items.json")
path_insert                     = "repository\\querys\\operation\\insert\\items.sql"
path_download_img               = os.path.join("data", "processed", "dota2", "items", "img")

# Read the JSON file containing game versions
if os.path.exists(path_file):
    with open(path_file, "r", encoding="utf-8") as file:
        items                   = json.load(file)

# Read the SQL query template from the file
with open(path_insert, "r", encoding="utf-8") as file:
    query_insert        = file.read()

if items:
    # INSERINDO OS ITENS NO BANCO DE DADOS
    for item_name in tqdm(list(items), desc="Inserting Items the Game"):
        item                    = items[item_name]
        sql_template            = Template(query_insert)
        path_download_img_final = os.path.join(path_download_img,"{}.png".format(os.path.splitext(os.path.basename(item["img"]))[0]))
        rendered_sql            = sql_template.render(
            ID_ITEM             = item["id"]                                    if "id" in item else -1,
            NAME                = item_name,
            DNAME               = item["dname"].replace("'", "").strip()        if "dname" in item else "",
            IMG                 = path_download_img_final,
            QUAL                = item["qual"].replace("'", "").strip()         if "qual" in item else None,
            COST                = item["cost"]                                  if "cost" in item else None,
            DMG_TYPE            = item["dmg_type"].replace("'", "").strip()     if "dmg_type" in item else None,
            NOTES               = item["notes"].replace("'", "").strip()        if "notes" in item else None,
            MC                  = item["mc"]                                    if "mc" in item and item["mc"] else None,
            HC                  = item["hc"]                                    if "hc" in item and item["hc"] else None,
            CD                  = item["cd"]                                    if "cd" in item and item["cd"] else None,
            LORE                = item["lore"].replace("'", "").strip()         if "lore" in item else None,
            CREATED             = item["created"]                               if "created" in item else None,
            CHARGES             = item["charges"]                               if "charges" in item and item["charges"] else None,
            ID_GAME             = 1
        ).replace("'None'", "NULL").replace("None", "NULL")\
        .replace("'False'", "FALSE").replace("False", "FALSE")\
        .replace("'True'", "TRUE").replace("True", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
        except Exception as e:
            print(f"Error inserting video data: {e}")
            db_manager.connection.rollback()
        
        # BAIXANDO IMAGEM DO HERÓI
        status, path    = FD.download((repository_img + item["img"]), path_download_img_final)


# BAIXANDO IMAGE DAS ABILIDADES

In [ ]:
repository = ["https://cdn.cloudflare.steamstatic.com/apps/dota2/images/dota_react/abilities/{}.png"]

# BAIXANDO IMAGE DOS TEAMS

In [ ]:
repository = [
    "https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/{}.png",
    "https://cdn.steamusercontent.com/ugc/{}/{}/" # 2909225722380320  5C6EFC9004093ED29E9B48242DE79418BDDBFE30
]